## Utilities to load dataset and show images and labels.

In [133]:
from pathlib import Path
from typing import Tuple

import cv2
import numpy as np

def load_dataset(dataset_dir_path: Path) -> Tuple[np.ndarray, np.ndarray]:
    print("Loading dataset")
    x, y = [], []
    for i, class_dir in enumerate(sorted(dataset_dir_path.iterdir())):
        for file in class_dir.iterdir():
            img_file = cv2.imread(str(file))  # TODO: Check if reading as GRAYSCALE image isn't better
            x.append(img_file)
            y.append(i)

    return np.asarray(x), np.asarray(y)


def show_images_and_labels(images, labels):
    for image, label in zip(images, labels):
        cv2.imshow("image", image)
        print(label)
        cv2.waitKey()

## Function to process the image.

In [145]:
def data_processing(x: np.ndarray) -> np.ndarray:
    print("Processing data")
    images_resized = []
    for image in x:
        image_resized = cv2.resize(image, (1200, 1000))  # TODO: Check smaller sizes
        images_resized.append(image_resized)

    return np.asarray(images_resized)

## Function to create vocabulary model

In [81]:
from sklearn import cluster
import pickle

def create_vocab_model(train_descriptors, nb_words=64):
    print("Creating vocab model")
    kmeans = cluster.KMeans(n_clusters=nb_words, random_state=42)
    kmeans.fit(train_descriptors)
    # pickle.dump(kmeans, open("vocab_model.p", "wb"))
    return kmeans

## Function that converts image descriptors to histogram based on vocabulary model

In [49]:
def convert_descriptor_to_histogram(descriptors, vocab_model, normalize=True) -> np.ndarray:
    features_words = vocab_model.predict(descriptors)  # KMeans returns indexes of words
    histogram = np.zeros(vocab_model.n_clusters, dtype=np.float32)
    unique, counts = np.unique(features_words, return_counts=True)
    histogram[unique] += counts
    if normalize:
        histogram /= histogram.sum()
    return histogram

## Function that applies feature transform on image, and then create histogram from image descriptors 

In [50]:
def apply_feature_transform(data: np.ndarray,
                            feature_detector_descriptor,
                            vocab_model
                            ) -> np.ndarray:
    print("Applying future transform")
    data_transformed = []
    for image in data:
        keypoints, image_descriptors = feature_detector_descriptor.detectAndCompute(image, None)
        bow_features_histogram = convert_descriptor_to_histogram(image_descriptors, vocab_model)
        data_transformed.append(bow_features_histogram)
    return np.asarray(data_transformed)

# Main part of training
## Loading dataset

In [146]:
import os

data_path = Path('drive/MyDrive/train_poznan_classification/')  # You can change the path here
data_path = os.getenv('DATA_PATH', data_path)  # Don't change that line
images, labels = load_dataset(data_path)

Loading dataset


## Processing images

In [147]:
  images = data_processing(images)


Processing data


## Spliting training data

In [148]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, train_size=0.8,
                                                                            random_state=42, stratify=labels)

## Creating feature descriptor

In [149]:
feature_detector_descriptor = cv2.AKAZE_create()  # TODO: Check different parameters


## Creating new vocabulary model

In [150]:
  train_descriptors = []
  for image in train_images:
      # TODO: Moze sprawdz maske binarna na srodku obrazu
      # Drugim argumentem może być maska binarna, która służy do zawężenia obszaru z którego
      # uzyskujemy punkty kluczowe/deskryptor – jako, że nam akurat zależy na całym obrazie,
      # podaliśmy tam wartość None.
      for descriptor in feature_detector_descriptor.detectAndCompute(image, None)[1]:
          train_descriptors.append(descriptor)
  vocab_model = create_vocab_model(train_descriptors)

Creating vocab model


## Use created vocabulary model to apply feature transform on train images and test images

In [151]:
# with Path('vocab_model.p').open('rb') as vocab_file:  # Don't change the path here
#     vocab_model = pickle.load(vocab_file)

X_train = apply_feature_transform(train_images, feature_detector_descriptor, vocab_model)
y_train = train_labels

X_test = apply_feature_transform(test_images, feature_detector_descriptor, vocab_model)
y_test = test_labels

Applying future transform
Applying future transform


In [152]:
eval_path = Path('drive/MyDrive/test_poznan_classification/')  # You can change the path here
eval_images, eval_labels = load_dataset(eval_path)
eval_images = data_processing(eval_images)
X_eval = apply_feature_transform(eval_images, feature_detector_descriptor, vocab_model)
y_eval = eval_labels


Loading dataset
Processing data
Applying future transform


## Create classifier

In [157]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# classifier = DecisionTreeClassifier()
classifier = svm.SVC()

## Search for the best params

In [158]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
param_grid = {
    "C": [1, 1.5, 2, 2.5, 5],
    "kernel": ["rbf", "poly", "sigmoid"],
    "degree": [3, 4, 5, 6],
    "gamma": ["scale", "auto"]
}
k_fold = KFold(n_splits=5)

grid_search = GridSearchCV(classifier, param_grid, cv=k_fold)
grid_search.fit(X_train, y_train)

print(grid_search.score(X_test, y_test))
print(grid_search.score(X_eval, y_eval))
print(grid_search.best_params_)

0.9642857142857143
0.85
{'C': 2.5, 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf'}


## Train the classifier

In [154]:
classifier.fit(X_train, y_train)
# print(classifier.score(X_train, y_train))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [155]:
print(classifier.score(X_test, y_test))
print(classifier.score(X_eval, y_eval))

0.9642857142857143
0.85


## Save vocab model and classifier

In [156]:
pickle.dump(vocab_model, open("vocab_model.p", "wb"))
pickle.dump(classifier, open("clf.p", "wb"))
